In [ ]:
# Based on 6250 distinct labels (625 labels and 10 synonyms per label retrieved and consolidated) we would get approx. 1,9 Mio. possible activity label pair combinations. 
# Therefore strucuture approach 'semantic search pair sampling' based on idea from Thakur et al. 2021 to retrieve high scored synonyms to reduce computational overhead.   

In [15]:
from torch.utils.data import DataLoader
from sentence_transformers import models, losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from datetime import datetime
import logging
import csv
import torch
import tqdm
import sys
import math
import gzip
import os

In [229]:
import pandas as pd

source_path = '' #specify source path of csv file
output_path = '' # specify path where pairs should be saved
csv = pd.read_csv(source_path, sep=";")

In [230]:
sentences = list(csv['augmented_labels'])

In [231]:
#### Top k similar sentences to be retrieved ####
#### Larger the k, bigger the silver dataset ####
top_k = 6

silver_data = []

sent2idx = {sentence: idx for idx, sentence in enumerate(sentences)} # storing id and sentence in dictionary
#duplicates = set((sent2idx[data.texts[0]], sent2idx[data.texts[1]]) for data in gold_samples) # not to include gold pairs of sentences again

In [232]:
# Loading the pretrained version of BERT 'all-mpnet-base-v2'

semantic_model_name = 'all-mpnet-base-v2' 

In [233]:
# encoding all unique sentences present in the training dataset using semantic model
logging.info("Encoding unique sentences with semantic search model: {}".format(semantic_search_model))
embeddings = semantic_search_model.encode(sentences, batch_size=batch_size, convert_to_tensor=True)

In [234]:
# retrieving top-k sentences given a sentence from the dataset
#progress = tqdm.tqdm(unit="docs", total=len(sent2idx))
for idx in range(len(sentences)):
    sentence_embedding = embeddings[idx]
    cos_scores = util.cos_sim(sentence_embedding, embeddings)[0]
    cos_scores = cos_scores.cpu()
    #progress.update(1)

    #torch.topk to find the highest 10 scores
    top_results = torch.topk(cos_scores, k=top_k+1)
    
    for score, iid in zip(top_results[0], top_results[1]):
        if iid != idx: #and (iid, idx) not in duplicates:
            silver_data.append((sentences[idx], sentences[iid]))
            #duplicates.add((idx,iid))

#progress.reset()
#progress.close()

In [236]:
# Converting the zipped list into pandas dataframe and dropping out duplicates
# since the augmenting enginge generates the same synonyms sometimes
df_silver = pd.DataFrame(silver_data, columns = ['label1','label2'])
df_silver_without_duplicates = df_silver.drop_duplicates()
df_silver_without_duplicates

In [238]:
# Saving dataframe without duplicates as .csv file

header = ["label1", "label2"]
#df.to_csv('output.csv', columns = header)

df_silver_without_duplicates.to_csv(output_path, sep=';', columns = header, header=True, index=False)# sep=';')